* 【hp.obo】①id/②name（表現型）
* 【phenotype.hpoa】 ③DiseaseName(疾患名、正式名、専門用語)/④HPO_ID

* 水色:②（ターゲット）
* 白:②
* ピンク:③

In [1]:
!git

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone             Clone a repository into a new directory
   init              Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add               Add file contents to the index
   mv                Move or rename a file, a directory, or a symlink
   restore           Restore working tree files
   rm                Remove files from the working tree and from the index
   sparse-checkout   Initialize and modify the sparse-checkout

examine the history and state (see also: git help revisio

In [3]:
!git add -A

In [6]:
!git commit

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .ipynb_checkpoints/hp.obo_phenotype.hpoa_ver1_id_name(Graphviz)-Copy1-checkpoint.ipynb
	modified:   hp.obo_phenotype.hpoa_ver1_id_name(Graphviz)-Copy1.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [1]:
# ライブラリ読み込み
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pronto
import copy
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from graphviz import Digraph
from IPython.display import Image

In [2]:
target_HP = "HP:0001249"

## ファイル読み込み

In [3]:
# 'phenotype.hpoa'読み込み

phenotype_df = pd.read_csv("phenotype.hpoa", header=4, sep="\t", engine="python")

# 列名#DatabaseIDの#をとる
phenotype_df = phenotype_df.rename(columns={'#DatabaseID': "DatabaseID"})

#DatabaseID が OMIM で始まるものに絞る
phenotype_df = phenotype_df[phenotype_df["DatabaseID"].str.contains("OMIM")]
phenotype_df.head(3)

,DatabaseID,DiseaseName,Qualifier,HPO_ID,Reference,Evidence,Onset,Frequency,Sex,Modifier,Aspect,Biocuration
0,OMIM:618850,Hypervalinemia or hyperleucine-isoleucinemia,NaN,HP:0010913,PMID:25653144,PCS,NaN,1/1,NaN,NaN,P,HPO:probinson[2020-07-23];HPO:probinson[2020-0...
1,OMIM:618850,Hypervalinemia or hyperleucine-isoleucinemia,NaN,HP:0000007,PMID:25653144,PCS,NaN,NaN,NaN,NaN,I,HPO:probinson[2020-07-23];HPO:probinson[2020-0...
2,OMIM:618850,Hypervalinemia or hyperleucine-isoleucinemia,NaN,HP:0002922,PMID:25653144,PCS,NaN,1/1,NaN,NaN,P,HPO:probinson[2020-07-23];HPO:probinson[2020-0...


In [4]:
# 'hp.obo'読み込み
#ont = pronto.Ontology('hp.obo')

ont = pronto.Ontology('https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo')

<ipython-input-4-c3a8f71683be>:4: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  ont = pronto.Ontology('https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo')


## スーパークラス

In [5]:
# ターム(id)を抽出

sup_target_HP_list = list(ont[target_HP].superclasses()) # 自分を入れる場合は引数不要（デフォルト）
sup_target_HP_list

[Term('HP:0001249', name='obsolete Mental retardation, in some'),
 Term('HP:0011446', name='Abnormality of higher mental function'),
 Term('HP:0012759', name='Neurodevelopmental abnormality'),
 Term('HP:0012638', name='Abnormal nervous system physiology'),
 Term('HP:0000707', name='Abnormality of the nervous system'),
 Term('HP:0000118', name='Phenotypic abnormality'),
 Term('HP:0000001', name='All')]

In [6]:
# 抽出したスーパークラスのidごとに、phenotype_dfからデータを取り出してリスト(relation_list)にする

relation_list = []

for i in range(len(sup_target_HP_list)):
    relation_list_a = phenotype_df[phenotype_df["HPO_ID"]==sup_target_HP_list[i].id][["DatabaseID","DiseaseName","HPO_ID"]]
    relation_list_a =relation_list_a.values.tolist()
    relation_list.append(relation_list_a)

In [7]:
# 【確認】relation_list
for i in range(len(relation_list)):
        print('relation_list',[i], len(relation_list[i]), sup_target_HP_list[i])

relation_list [0] 1324 Term('HP:0001249', name='obsolete Mental retardation, in some')
relation_list [1] 0 Term('HP:0011446', name='Abnormality of higher mental function')
relation_list [2] 9 Term('HP:0012759', name='Neurodevelopmental abnormality')
relation_list [3] 0 Term('HP:0012638', name='Abnormal nervous system physiology')
relation_list [4] 53 Term('HP:0000707', name='Abnormality of the nervous system')
relation_list [5] 0 Term('HP:0000118', name='Phenotypic abnormality')
relation_list [6] 0 Term('HP:0000001', name='All')


In [8]:
# target_HPのnameも取得しておく
target_HP_name = sup_target_HP_list[0].name
target_HP_name

'obsolete Mental retardation, in some'

In [9]:
# 抽出したタームを階層ごとに分ける
# 1. 全階層を入れる箱と、その中に入れる階層ごとの箱を作成
# 2. 階層ごとの箱の中身を整理

# 3. 全階層の箱の要素を
# "superclass_"+str(i+1)+"layer_list_id階層ごとの箱に振り分ける


# 1. 全階層を入れる箱と、その中に入れる階層ごとの箱を作成

superclass_layer_list =[] # 全階層を入れる箱を用意

for i in range(1,10000):
    
    # 階層ごとの箱＝スーパークラスのdistanceごとのタームリスト　とする
    exec("superclass_"+str(i)+"layer_list" + " = list(ont[target_HP].superclasses(distance="+str(i)+"))")
    
    # 階層の深さを知る
    if list(ont[target_HP].superclasses(distance=i-1)) == list(ont[target_HP].superclasses(distance=i)):
        print("Max depth:" + str(i-1))         
        break;
    
     # 全階層用の箱に、階層ごとの箱を入れていく
    superclass_layer_list.append("superclass_"+str(i)+"layer_list")
    
max_len=i

Max depth:5


In [10]:
# 2. 階層ごとの箱の中身を整理

# distance=2のスーパークラスにはdistance=1のスーパークラスも含まれる。
# その階層だけが入った箱にするため、1つ前のdistanceのスーパークラスの分を引くことで
# その階層だけのデータが残る、という設定にする。

for i in range(max_len-2,0,-1): # 降順
    print(i)
    # 1つ前のdistanceのスーパークラスの分を引きその階層だけのデータにする
    exec("superclass_"+str(i+1)+"layer_list" "= set("+superclass_layer_list[i]+") - set(" + superclass_layer_list[i-1]+ ")")

4
3
2
1


In [11]:
# ':'を削除しておく
target_HP_rep=target_HP.replace(':', '')

In [12]:
superclass_layer_list

['superclass_1layer_list',
 'superclass_2layer_list',
 'superclass_3layer_list',
 'superclass_4layer_list',
 'superclass_5layer_list']

In [13]:
# 3. 全階層の箱(superclass_layer_list)の要素を
# "superclass_"+str(i+1)+"layer_list_id の箱に振り分ける

# 全階層の箱から階層ごとに要素を取得
for i,superclasslist in enumerate(superclass_layer_list):
    
    # superclasslistから"superclass_"+str(i+1)+"layer_list_id　という箱を作る
    exec("superclass_"+str(i+1)+"layer_list_id = eval(superclasslist)")
    
    # idとnameを合体
    exec("superclass_"+str(i+1)+"layer_list_id = [s.id + s.name for s in superclass_"+ str(i+1) +"layer_list_id]")
    
    # ":"を削除
    exec("superclass_"+str(i+1)+"layer_list_id = [s.replace(':', '') for s in superclass_" +str(i+1)+"layer_list_id]")
    
    # 逆順取得
    exec("superclass_"+str(i+1)+"layer_list_id = list(reversed(superclass_"+str(i+1)+"layer_list_id))")


In [14]:
# 【違い確認】superclass_5layer_list　と　superclass_5layer_list_idsuperclass＿5layer＿list
print("superclass_5layer_list   : ", superclass_5layer_list)
print("superclass_5layer_list_id: ", superclass_5layer_list_id) # 【確認】 id,nameが合体し":"が消えているか

superclass_5layer_list   :  {Term('HP:0000001', name='All')}
superclass_5layer_list_id:  ['HP0000001All']


In [15]:
#グラフライブラリ
from graphviz import Digraph
import networkx as nx

G = Digraph(format="png") # graphviz

In [16]:
# ho.oboとphenotype.hpoa両方にグラフ設定

# hp.obo
# 1. ノードの設定
# 2. エッジの設定(ターム,name表示)


# 1. ノードの設定

def connected_judge(HP_1,HP_2,replaced=True,name_plus=True):
    # HP_1 parentid
    # HP_2 parentid
    if replaced:
        HP_1 = HP_1.replace('HP', 'HP:')
        HP_2 = HP_2.replace('HP', 'HP:')
    if name_plus:
        HP_1 = HP_1[:10]
        HP_2 = HP_2[:10]
    HP_2_list = list(ont[HP_2].superclasses(distance=1))
    HP_2_list = [s.id for s in HP_2_list]
    return HP_1 in HP_2_list

In [17]:
# 2. エッジの設定

for superclasslist in superclass_layer_list:
    
    tmplist = eval(superclasslist+"_id") 
    
    # superclass_1layer_listとそれ以外に分けて処理
    if superclasslist == 'superclass_1layer_list':
        for hp_node_ind_1 in range(len(tmplist)-1):
            G.edge(tmplist[hp_node_ind_1][:], target_HP_rep+target_HP_name) # nameのみ表示は"[9:],target_HP_name"
            before_list = copy.copy(tmplist)            
    else:
        for hp_id_super_class in tmplist:
            for hp_id_sub_class_candidate in before_list:
                if connected_judge(hp_id_super_class,hp_id_sub_class_candidate,replaced=True):
                    G.edge(hp_id_super_class[:], hp_id_sub_class_candidate[:]) # nameのみ表示は[9:]
        before_list = copy.copy(tmplist)

In [18]:
# phenotype.hpoa

# 「:」削除
for r in range(len(relation_list)):
    for i,y in enumerate(relation_list[r]):        
        y[0] = y[0].replace(':', '') # y[0] OMIMから削除
        y[2] = y[2].replace(':', '') # y[2] HPO_ID から削除

In [19]:
sup_target_HP_list

[Term('HP:0001249', name='obsolete Mental retardation, in some'),
 Term('HP:0011446', name='Abnormality of higher mental function'),
 Term('HP:0012759', name='Neurodevelopmental abnormality'),
 Term('HP:0012638', name='Abnormal nervous system physiology'),
 Term('HP:0000707', name='Abnormality of the nervous system'),
 Term('HP:0000118', name='Phenotypic abnormality'),
 Term('HP:0000001', name='All')]

In [20]:
# phenotype.hpoaのDiseaseNameについて、5個づつ四角枠でグラフ表示

for one_layer,base_layer in zip(relation_list,sup_target_HP_list):
    for i,one_element in enumerate(one_layer):
        
        # ノード
        G.node(one_element[1].replace('%','\%'), shape='box', style="filled", fillcolor="mistyrose") 
        
        # エッジ（hp.oboとphenotype.hpoaをつなぐ）
        G.edge(one_element[1].replace('%','\%'), base_layer.id.replace(":", '')+base_layer.name) # nameのみ表示はbase_layer.name
        
        #  最初の「%」を表示させる
        print(one_element[1].replace('%','\%'))
        
        if i > 5: # 表示5個まで
            break;

127350 DYSCHONDROSTEOSIS AND NEPHRITIS
#614249 MENTAL RETARDATION, AUTOSOMAL RECESSIVE 18; MRT18
ARGININOSUCCINIC ACIDURIA
OROFACIODIGITAL SYNDROME III
METHYLMALONIC ACIDURIA AND HOMOCYSTINURIA, CBLD TYPE
MENTAL RETARDATION WITH OPTIC ATROPHY, DEAFNESS, AND SEIZURES
WOLFRAM SYNDROME, MITOCHONDRIAL FORM
Immunodeficiency 38
MARFAN LIPODYSTROPHY SYNDROME
#605751 SEIZURES, BENIGN FAMILIAL INFANTILE, 2; BFIS2;;CONVULSIONS, BENIGN FAMILIAL INFANTILE, 2; BFIC2
#602066 CONVULSIONS, FAMILIAL INFANTILE, WITH PAROXYSMAL CHOREOATHETOSIS;ICCA;;INFANTILE CONVULSIONS AND PAROXYSMAL CHOREOATHETOSIS, FAMILIAL;;ICCA SYNDROME;;PAROXYSMAL KINESIGENIC DYSKINESIA WITH INFANTILE CONVULSIONS; PKD/IC
#607745 SEIZURES, BENIGN FAMILIAL INFANTILE, 3; BFIS3;;CONVULSIONS, BENIGN FAMILIAL INFANTILE, 3; BFIC3;;SEIZURES, BENIGN FAMILIAL NEONATAL-INFANTILE; BFNIS
\%601764 SEIZURES, BENIGN FAMILIAL INFANTILE, 1; BFIS1;;CONVULSIONS, BENIGN FAMILIAL INFANTILE, 1; BFIC1
#615158 MITOCHONDRIAL COMPLEX III DEFICIENCY, NUCLEAR

In [21]:
# target_HP_nameに色付け
G.node(target_HP_rep+target_HP_name, shape="ellipse", style="filled", fillcolor="lightblue2")

In [22]:
G.render()
G.view()
#!dot -T png Digraph.gv > test.png
#Image('test.png')

'Digraph.gv.png'

## サブクラス

In [23]:
# 以下工程はスーパークラスと同じ

# ターム(id)を抽出
sub_target_HP_list = list(ont[target_HP].subclasses()) [1:] #  [1:]でtarget_HP削除
sub_target_HP_list

[Term('HP:0001256', name='Intellectual disability, mild'),
 Term('HP:0002187', name='Intellectual disability, profound'),
 Term('HP:0002342', name='Intellectual disability, moderate'),
 Term('HP:0006887', name='Intellectual disability, progressive'),
 Term('HP:0006889', name='Intellectual disability, borderline'),
 Term('HP:0010864', name='Intellectual disability, severe')]

In [24]:
# 抽出したサブクラスのターム(id)ごとに、phenotype_dfからデータを取り出してリスト(sub_relation_list)にする

sub_relation_list = []

for i in range(len(sub_target_HP_list)):
    sub_relation_list_a = phenotype_df[phenotype_df["HPO_ID"]==sub_target_HP_list[i].id][["DatabaseID","DiseaseName","HPO_ID"]]
    sub_relation_list_a =sub_relation_list_a.values.tolist()
    sub_relation_list.append(sub_relation_list_a)

In [25]:
sub_relation_list_a

[['OMIM:212720',
  '#212720 MARTSOLF SYNDROME;;CATARACT-MENTAL RETARDATION-HYPOGONADISM',
  'HP:0010864'],
 ['OMIM:614207',
  '#614207 HYPERPHOSPHATASIA WITH MENTAL RETARDATION SYNDROME 3; HPMRS3;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 17; MRT17;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 21; MRT21',
  'HP:0010864'],
 ['OMIM:256150', 'NEPHROSIALIDOSIS', 'HP:0010864'],
 ['OMIM:609241', 'SCHINDLER DISEASE, TYPE I', 'HP:0010864'],
 ['OMIM:614342', 'MENTAL RETARDATION, AUTOSOMAL RECESSIVE 30', 'HP:0010864'],
 ['OMIM:616763',
  'LEUKODYSTROPHY AND ACQUIRED MICROCEPHALY WITH OR WITHOUT DYSTONIA',
  'HP:0010864'],
 ['OMIM:611603', 'LISSENCEPHALY 3', 'HP:0010864'],
 ['OMIM:610680',
  '610680 HOLOPROSENCEPHALY, RECURRENT INFECTIONS, AND MONOCYTOSIS',
  'HP:0010864'],
 ['OMIM:300209',
  '#300209 SIMPSON-GOLABI-BEHMEL SYNDROME, TYPE 2;;SGBS2',
  'HP:0010864'],
 ['OMIM:613150',
  '#613150 MUSCULAR DYSTROPHY-DYSTROGLYCANOPATHY (CONGENITAL WITH BRAIN AND EYEANOMALIES), TYPE A, 2; MDDGA2;;WALKER-WARBUR

In [26]:
# 【確認】sub_relation_list
for i in range(len(sub_relation_list)):
        print('sub_relation_list',[i], len(sub_relation_list[i]), sub_target_HP_list[i])

sub_relation_list [0] 154 Term('HP:0001256', name='Intellectual disability, mild')
sub_relation_list [1] 61 Term('HP:0002187', name='Intellectual disability, profound')
sub_relation_list [2] 51 Term('HP:0002342', name='Intellectual disability, moderate')
sub_relation_list [3] 65 Term('HP:0006887', name='Intellectual disability, progressive')
sub_relation_list [4] 5 Term('HP:0006889', name='Intellectual disability, borderline')
sub_relation_list [5] 155 Term('HP:0010864', name='Intellectual disability, severe')


In [27]:
# 抽出したタームを階層ごとに分ける
# 1. 全階層を入れる箱と、その中に入れる階層ごとの箱を作成
# 2. 階層ごとの箱の中身を整理
# 3. 全階層の箱の要素を
# "subclass_"+str(i+1)+"layer_list_id階層ごとの箱に振り分ける


# 1. 全階層を入れる箱と、その中に入れる階層ごとの箱を作成

subclass_layer_list =[] # 全階層を入れる箱を用意

for i in range(1,10000):
    
    # 階層ごとの箱＝サブクラスのdistanceごとのタームリスト　とする
    exec("subclass_"+str(i)+"layer_list" + " = list(ont[target_HP].subclasses(distance="+str(i)+"))")
    
    # 階層の深さ
    if list(ont[target_HP].subclasses(distance=i-1)) == list(ont[target_HP].subclasses(distance=i)):
        print("Max depth:" + str(i-1))         
        break;
        
    # 全階層用の箱に、階層ごとの箱を入れていく
    subclass_layer_list.append("subclass_"+str(i)+"layer_list")

max_len=i

Max depth:1


In [28]:
# 2. 階層ごとの箱の中身を整理

# distance=2のサブクラスにはdistance=1のスーパークラスも含まれる。
# その階層だけが入った箱にするため、1つ前のdistanceのサブクラスの分を引くことで
# その階層だけのデータが残る、という設定にする。

for i in range(max_len-2,0,-1): # 降順
    print(i)
    # 1つ前のdistanceのサブクラスの分を引きその階層だけのデータにする
    exec("subclass_"+str(i+1)+"layer_list" "= set("+subclass_layer_list[i]+") - set(" + subclass_layer_list[i-1]+ ")")

In [29]:
# 3. 全階層の箱(subclass_layer_list)の要素を
# "subclass_"+str(i+1)+"layer_list_id の箱に振り分ける

# 全階層の箱から階層ごとに要素を取得
for i,subclasslist in enumerate(subclass_layer_list):
    
    # subclasslistから"subclass_"+str(i+1)+"layer_list_id　という箱を作る
    exec("subclass_"+str(i+1)+"layer_list_id = eval(subclasslist)")
    # idとnameを合体
    exec("subclass_"+str(i+1)+"layer_list_id = [s.id + s.name for s in subclass_"+ str(i+1) +"layer_list_id]")
    # ":"を削除
    exec("subclass_"+str(i+1)+"layer_list_id = [s.replace(':', '') for s in subclass_" +str(i+1)+"layer_list_id]")
    # 逆順取得
    exec("subclass_"+str(i+1)+"layer_list_id = list(reversed(subclass_"+str(i+1)+"layer_list_id))")

In [30]:
# ho.oboとphenotype.hpoa両方にグラフ設定


# hp.obo
# 1. ノードの設定
# 2. エッジの設定(ターム,name表示)


# ノードの設定

def connected_judge(HP_1,HP_2,replaced=True,name_plus=True):
    # HP_1 parentid
    # HP_2 parentid
    if replaced:
        HP_1 = HP_1.replace('HP', 'HP:')
        HP_2 = HP_2.replace('HP', 'HP:')
    if name_plus:
        HP_1 = HP_1[:10]
        HP_2 = HP_2[:10]
    HP_2_list = list(ont[HP_2].subclasses(distance=1))
    HP_2_list = [s.id for s in HP_2_list]
    return HP_1 in HP_2_list

# エッジの設定

for subclasslist in subclass_layer_list:
    tmplist = eval(subclasslist+"_id") 
    
    # superclass_1layer_listとそれ以外に分けて処理
    if subclasslist == 'subclass_1layer_list':
        for hp_node_ind_1 in range(len(tmplist)-1):
            G.edge(tmplist[hp_node_ind_1][:],target_HP_rep+target_HP_name) # nameのみ表示は"[9:],target_HP_name"
            before_list = copy.copy(tmplist)            
    else:
        for hp_id_super_class in tmplist:
            for hp_id_sub_class_candidate in before_list:
                if connected_judge(hp_id_super_class,hp_id_sub_class_candidate,replaced=True):
                    G.edge(hp_id_super_class[:],hp_id_sub_class_candidate[:])# nameのみ表示[9:]
        before_list = copy.copy(tmplist)

In [31]:
# エッジの設定

for subclasslist in subclass_layer_list:
    tmplist = eval(subclasslist+"_id") 
    
    # superclass_1layer_listとそれ以外に分けて処理
    if subclasslist == 'subclass_1layer_list':
        for hp_node_ind_1 in range(len(tmplist)-1):
            G.edge(target_HP_rep+target_HP_name, tmplist[hp_node_ind_1][:]) # nameのみ表示は"[9:],target_HP_name"
            before_list = copy.copy(tmplist)            
    else:
        for hp_id_sub_class in tmplist:
            for hp_id_sub_class_candidate in before_list:
                if connected_judge(hp_id_sub_class, hp_id_sub_class_candidate,replaced=True):
                    G.edge(hp_id_sub_class[:], hp_id_sub_class_candidate[:])# nameのみ表示[9:]
        before_list = copy.copy(tmplist)

In [32]:
before_list

['HP0010864Intellectual disability, severe',
 'HP0006889Intellectual disability, borderline',
 'HP0006887Intellectual disability, progressive',
 'HP0002342Intellectual disability, moderate',
 'HP0002187Intellectual disability, profound',
 'HP0001256Intellectual disability, mild',
 'HP0001249obsolete Mental retardation, in some']

In [33]:
# phenotype.hpoa

# 「:」がエラーとなるため削除
for r in range(len(sub_relation_list)):
    for i,y in enumerate(sub_relation_list[r]):
        y[0] = y[0].replace(':', '') # y[0] OMIMから削除
        y[2] = y[2].replace(':', '') # y[2] HPO_ID から削除

In [34]:
sub_relation_list

[[['OMIM604004',
   'MEGALENCEPHALIC LEUKOENCEPHALOPATHY WITH SUBCORTICAL CYSTS',
   'HP0001256'],
  ['OMIM617107', 'THAUVIN-ROBINET-FAIVRE SYNDROME', 'HP0001256'],
  ['OMIM614207',
   '#614207 HYPERPHOSPHATASIA WITH MENTAL RETARDATION SYNDROME 3; HPMRS3;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 17; MRT17;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 21; MRT21',
   'HP0001256'],
  ['OMIM233430',
   'GONADAL DYSGENESIS, XY TYPE, WITH ASSOCIATED ANOMALIES',
   'HP0001256'],
  ['OMIM616738',
   'Radioulnar synostosis with amegakaryocytic thrombocytopenia 2',
   'HP0001256'],
  ['OMIM617308', 'BILE ACID SYNTHESIS DEFECT, CONGENITAL, 6', 'HP0001256'],
  ['OMIM617763',
   'SHORT STATURE, HEARING LOSS, RETINITIS PIGMENTOSA, AND DISTINCTIVE FACIES; SHRF',
   'HP0001256'],
  ['OMIM617757', 'JOUBERT SYNDROME 32; JBTS32', 'HP0001256'],
  ['OMIM615866', 'MENTAL RETARDATION, AUTOSOMAL DOMINANT 27', 'HP0001256'],
  ['OMIM102150', 'ACROMEGALOID FACIAL APPEARANCE SYNDROME', 'HP0001256'],
  ['OMIM300373', 'O

In [35]:
# phenotype.hpoaのDiseaseNameについて、5個づつ四角枠でグラフ表示

for one_layer,base_layer in zip(sub_relation_list,sub_target_HP_list):
    for i,one_element in enumerate(one_layer):
        
        # ノード設定　さらに最初の「%」も表示
        G.node(one_element[1].replace('%','\%'), shape='box', style="filled", fillcolor="mistyrose") 
        
        # エッジ（hp.oboとphenotype.hpoaをつなぐ）
        # ↓target_HPとのエッジはスーパークラスで実行しているので不要 
        # G.edge(target_HP_rep+target_HP_name, one_element[1].replace('%','\%')) # nameのみ表示はtarget_HP_name
        G.edge(base_layer.id.replace(":", '')+base_layer.name, one_element[1].replace('%','\%')) # nameのみ表示はbase_layer.name
        
        print(one_element[1].replace('%','\%'))
        
        if i > 5: # 表示5個まで
            break;

MEGALENCEPHALIC LEUKOENCEPHALOPATHY WITH SUBCORTICAL CYSTS
THAUVIN-ROBINET-FAIVRE SYNDROME
#614207 HYPERPHOSPHATASIA WITH MENTAL RETARDATION SYNDROME 3; HPMRS3;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 17; MRT17;;MENTAL RETARDATION, AUTOSOMAL RECESSIVE 21; MRT21
GONADAL DYSGENESIS, XY TYPE, WITH ASSOCIATED ANOMALIES
Radioulnar synostosis with amegakaryocytic thrombocytopenia 2
BILE ACID SYNTHESIS DEFECT, CONGENITAL, 6
SHORT STATURE, HEARING LOSS, RETINITIS PIGMENTOSA, AND DISTINCTIVE FACIES; SHRF
#613150 MUSCULAR DYSTROPHY-DYSTROGLYCANOPATHY (CONGENITAL WITH BRAIN AND EYEANOMALIES), TYPE A, 2; MDDGA2;;WALKER-WARBURG SYNDROME OR MUSCLE-EYE-BRAIN DISEASE, POMT2-RELATED
Intellectual developmental disorder with seizures and language delay
Trichothiodystrophy 5, nonphotosensitive
Neurodevelopmental disorder with epilepsy, spasticity, and brain atrophy
Epileptic encephalopathy, early infantile, 81
MENTAL RETARDATION, AUTOSOMAL RECESSIVE 55
MICROCEPHALY, SEIZURES, SPASTICITY, AND BRAIN CALCIFI

In [36]:
G.render()
G.view()
#!dot -T png Digraph.gv > test.png
#Image('test.png')

'Digraph.gv.png'

In [37]:
!dot -T dot Digraph.gv -o Digraph.gv.dot
#dotに変換

In [38]:
stop

NameError: name 'stop' is not defined

In [ ]:
# networkxをjsonコンバート
>>> from networkx.readwrite import json_graph
>>> sample_G = nx.DiGraph([(1,2), (2,3), (3,4)])
>>> data= json_graph.tree_data(sample_G,root=1) #jsonにコンバート
>>> H = json_graph.tree_graph(data) # json読み込み
print("jsonフォーマット: ", data)

plt.subplot(1,2,1)
nx.draw_networkx(sample_G) # networkxで作成
plt.title("sample_G")

plt.subplot(1,2,2)
nx.draw_networkx(H) #sample_Gをjsonにコンバートし、networkxで可視化
plt.title("H")

In [ ]:
# dotファイルをnetworkxで読み込み
G_dot = nx.drawing.nx_agraph.read_dot('Digraph.gv.dot')
type(G_dot)

# グラフ作成
nx.draw_networkx(G_dot)

In [ ]:
# 【確認】ノード
pd.DataFrame(G_dot.nodes)

In [ ]:
# 【確認】エッジ
pd.DataFrame(G_dot.edges)

In [ ]:
# networkxをjsonコンバート
>>> from networkx.readwrite import json_graph
>>> #G = nx.DiGraph([(1,2)])
>>> data = json_graph.tree_graph(G_dot)

In [ ]:
G_dot

In [ ]:
#!dot -T png Digraph.gv > test.png
#Image('test.png')

In [ ]:
import json


# Call dumps on our list.
result = json.dumps(G.body)
# Display the JSON data.
print(result)
#print("Length:", len(result))

In [ ]:
G.body

In [ ]:
print(type(G.body))
print(type(G.source))

In [ ]:


import json
import urllib.request

# Let's use the Vega radial tree example as a basis of the visualization
# jsonファイルの読み込み
view = json.load(urllib.request.urlopen("https://vega.github.io/vega/examples/radial-tree-layout.vg.json"))

# First replace the default data with our own
view['data'][0].pop('url')
view['data'][0]['values'] = result
view['marks'][1]['encode']['enter']['tooltip'] = {"signal": "datum.desc"}
view['signals'][4]['value'] = 'cluster'

# Render the clustered tree
display({"application/vnd.vega.v5+json": view}, raw=True)

In [ ]:
import json
json_file = open('test.json', 'w')
json.dump(result, json_file)

In [ ]:
# ここから開始！
# http://ja.voidcc.com/question/p-vsqksqmm-gz.html
# https://showa-yojyo.github.io/notebook/python-networkx/drawing.html
# https://networkx.org/documentation/stable/reference/generated/networkx.convert.to_networkx_graph.html

In [ ]:
# dot→json　読み込み
import networkx as nx
from networkx.readwrite import json_graph

import sys
import json


dot_graph = nx.nx_pydot.read_dot("convert.dot")
print(json.dumps(json_graph.node_link_data(dot_graph)))

In [ ]:
 def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

In [ ]:
nx.to_dict_of_lists(G)

In [ ]:
## コンバート2: pydotplusm→networkx
# Perform the conversion to networkx
import pydotplus
import networkx

dotplus = pydotplus.graph_from_dot_data(G.source)


# if graph doesn't have multiedges, use dotplus.set_strict(true)
nx_graph = nx.nx_pydot.from_pydot(dotplus)
print(type(nx_graph)) # prints <class 'networkx.classes.multidigraph.MultiDiGraph'>

nx.draw_networkx(nx_graph)
plt.axis('off')
plt.show()

In [ ]:
# networkx.convert.to_networkx_graph
# https://networkx.org/documentation/stable/reference/generated/networkx.convert.to_networkx_graph.html


# sample
d = {0: {1: {"weight": 1}}} # dict-of-dicts single edge (0,1)
type(d)

In [ ]:
type(G)

In [ ]:
#G = nx.DiGraph(d)
G = nx.from_dict_of_dicts(d)


# If True and data is a dict_of_dicts, try to create a multigraph assuming dict_of_dict_of_lists.

nx.draw_networkx(G)
plt.axis('off')
plt.show()

In [ ]:
pydotplus.graph_from_dot_data(G.source)

In [ ]:
G = nx.from_dict_of_dicts(t)

In [ ]:
import networkx as nx
# networkxで読み込み
G = nx.DiGraph()
nx.add_path(G, [3, 5, 4, 1, 0, 2, 7, 8, 9, 6])
nx.add_path(G, [3, 0, 6, 4, 2, 7, 1, 9, 8, 5])
# PyGraphviz
nx.nx_agraph.view_pygraphviz(G, prog='fdp') 

In [ ]:
import networkx as nx
from IPython.display import SVG, display

# networkxで読み込み
G = nx.DiGraph()
nx.add_path(G, [3, 5, 4, 1, 0, 2, 7, 8, 9, 6])
nx.add_path(G, [3, 0, 6, 4, 2, 7, 1, 9, 8, 5])
# PyGraphviz（Graphviz)
svg = SVG(nx.nx_agraph.to_agraph(G).draw(prog='fdp', format='svg')) # SVG表示(Jupyter)
display(svg)